# 💬 Interfaz de Chat - MiniGPT Instructivo

**Tarea 5 - Proyecto Final de Doctorado en IA/NLP**

Este notebook crea una interfaz interactiva para el modelo usando:
- **Gradio**: Interfaz web moderna y fácil de usar
- **CLI**: Interfaz de línea de comandos alternativa

---

## 1. Setup

In [1]:
# Instalar dependencias
!pip install gradio transformers torch -q
print("✅ Dependencias instaladas")

✅ Dependencias instaladas


In [2]:
import torch
import gradio as gr
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Dispositivo: {device}")

🖥️ Dispositivo: cuda


In [3]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/MiniGPT_Doctoral'

Mounted at /content/drive


In [4]:
# Cargar modelo
model_path = f"{project_path}/miniGPT_final"

print("🔄 Cargando modelo...")
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

model = model.to(device)
model.eval()

print(f"✅ Modelo cargado: {sum(p.numel() for p in model.parameters()):,} parámetros")

🔄 Cargando modelo...
✅ Modelo cargado: 110,025,216 parámetros


---

## 2. Función de Generación

In [5]:
def generate_response(
    instruction: str,
    input_text: str = "",
    max_new_tokens: int = 200,
    temperature: float = 0.7,
    top_p: float = 0.9,
    top_k: int = 50,
    repetition_penalty: float = 1.1
) -> str:
    """
    Genera una respuesta para una instrucción dada.

    Args:
        instruction: La instrucción o pregunta del usuario
        input_text: Contexto adicional (opcional)
        max_new_tokens: Máximo de tokens a generar
        temperature: Creatividad (0.1=conservador, 1.0=creativo)
        top_p: Nucleus sampling
        top_k: Top-k sampling
        repetition_penalty: Penalización por repetición

    Returns:
        Respuesta generada por el modelo
    """
    # Formatear prompt
    if input_text:
        prompt = f"""### Instrucción:
{instruction}

### Entrada:
{input_text}

### Respuesta:
"""
    else:
        prompt = f"""### Instrucción:
{instruction}

### Respuesta:
"""

    # Tokenizar
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(device)

    # Generar
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=repetition_penalty,
        )

    # Decodificar
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraer solo la respuesta
    if "### Respuesta:" in full_response:
        response = full_response.split("### Respuesta:")[-1].strip()
    else:
        response = full_response

    # Limpiar respuesta
    if "### Instrucción:" in response:
        response = response.split("### Instrucción:")[0].strip()

    return response

# Test rápido
test = generate_response("¿Qué es Python?")
print(f"Test: {test[:200]}...")

Test: Los gatos son: 1. Python 2. Java 3. Java 4. Python 5. Python 6. Python 6. JavaScript 7. JavaScript 8. Python 12. Python 7. Python 10. Python 9. Python 11. Python 8. Python 9. Python de código HTML, Ja...


---

## 3. Interfaz CLI (Línea de Comandos)

In [6]:
def chat_cli():
    """
    Interfaz de chat por línea de comandos.
    """
    print("\n" + "="*60)
    print("🤖 MINIGPT INSTRUCTIVO - Chat CLI")
    print("="*60)
    print("Escribe tu pregunta o instrucción.")
    print("Comandos especiales:")
    print("  'salir' - Terminar el chat")
    print("  'config' - Ver/cambiar configuración")
    print("  'limpiar' - Limpiar pantalla")
    print("="*60 + "\n")

    # Configuración por defecto
    config = {
        "max_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9
    }

    while True:
        try:
            user_input = input("\n👤 Tú: ").strip()

            if not user_input:
                continue

            if user_input.lower() == 'salir':
                print("\n👋 ¡Hasta luego!")
                break

            if user_input.lower() == 'limpiar':
                os.system('clear' if os.name == 'posix' else 'cls')
                continue

            if user_input.lower() == 'config':
                print(f"\n⚙️ Configuración actual:")
                for k, v in config.items():
                    print(f"   {k}: {v}")
                continue

            # Generar respuesta
            print("\n🤖 MiniGPT: ", end="")
            response = generate_response(
                user_input,
                max_new_tokens=config["max_tokens"],
                temperature=config["temperature"],
                top_p=config["top_p"]
            )
            print(response)

        except KeyboardInterrupt:
            print("\n\n👋 ¡Hasta luego!")
            break

# Descomentar para ejecutar CLI
# chat_cli()

---

## 4. Interfaz Gradio (Web)

In [7]:
def gradio_generate(
    instruction: str,
    input_text: str,
    max_tokens: int,
    temperature: float,
    top_p: float,
    repetition_penalty: float
) -> str:
    """
    Wrapper para Gradio.
    """
    if not instruction.strip():
        return "⚠️ Por favor, escribe una instrucción."

    response = generate_response(
        instruction=instruction,
        input_text=input_text,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )

    return response

In [8]:
# Crear interfaz Gradio
with gr.Blocks(
    title="MiniGPT Instructivo",
    theme=gr.themes.Soft()
) as demo:

    # Header
    gr.Markdown("""
    # 🤖 MiniGPT Instructivo

    **Modelo GPT de 110M parámetros entrenado en español**

    Proyecto Final de Doctorado en IA/NLP

    ---
    """)

    with gr.Row():
        with gr.Column(scale=2):
            # Inputs
            instruction_input = gr.Textbox(
                label="📝 Instrucción",
                placeholder="Escribe tu pregunta o instrucción aquí...",
                lines=3
            )

            context_input = gr.Textbox(
                label="📄 Contexto adicional (opcional)",
                placeholder="Añade contexto si es necesario...",
                lines=2
            )

            # Output
            output = gr.Textbox(
                label="🤖 Respuesta",
                lines=8,
                interactive=False
            )

            # Buttons
            with gr.Row():
                submit_btn = gr.Button("🚀 Generar", variant="primary")
                clear_btn = gr.Button("🗑️ Limpiar")

        with gr.Column(scale=1):
            # Parámetros
            gr.Markdown("### ⚙️ Parámetros")

            max_tokens = gr.Slider(
                minimum=50, maximum=500, value=200, step=10,
                label="Máx. tokens"
            )

            temperature = gr.Slider(
                minimum=0.1, maximum=1.5, value=0.7, step=0.1,
                label="Temperatura (creatividad)"
            )

            top_p = gr.Slider(
                minimum=0.1, maximum=1.0, value=0.9, step=0.05,
                label="Top-p (nucleus sampling)"
            )

            rep_penalty = gr.Slider(
                minimum=1.0, maximum=2.0, value=1.1, step=0.1,
                label="Penalización repetición"
            )

    # Ejemplos
    gr.Markdown("### 📚 Ejemplos")
    gr.Examples(
        examples=[
            ["Explica qué es la inteligencia artificial en palabras simples.", ""],
            ["Escribe un poema corto sobre el mar.", ""],
            ["¿Cómo funciona una red neuronal?", ""],
            ["Resume el siguiente texto:", "El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender de los datos."],
            ["Traduce al inglés:", "Hola, ¿cómo estás?"],
            ["Clasifica el sentimiento del texto:", "Me encanta este producto, es increíble."],
        ],
        inputs=[instruction_input, context_input]
    )

    # Footer
    gr.Markdown("""
    ---

    **Información del modelo:**
    - Arquitectura: GPT-2 (Decoder-Only)
    - Parámetros: 110M
    - Dataset: ~57k instrucciones en español
    - Tokenizador: BPE personalizado (32k tokens)
    """)

    # Event handlers
    submit_btn.click(
        fn=gradio_generate,
        inputs=[instruction_input, context_input, max_tokens, temperature, top_p, rep_penalty],
        outputs=output
    )

    clear_btn.click(
        fn=lambda: ("", "", ""),
        outputs=[instruction_input, context_input, output]
    )

    # También permitir Enter para enviar
    instruction_input.submit(
        fn=gradio_generate,
        inputs=[instruction_input, context_input, max_tokens, temperature, top_p, rep_penalty],
        outputs=output
    )

print("✅ Interfaz Gradio creada")

✅ Interfaz Gradio creada


/tmp/ipython-input-3429813010.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(


In [ ]:
# Lanzar interfaz
demo.launch(
    share=True,  # Genera URL pública temporal
    debug=True
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://42c71ad65fb14271f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---

## 5. Interfaz de Chat Conversacional

In [ ]:
def chat_response(message: str, history: list) -> str:
    """
    Función para chat conversacional con historial.
    """
    # Generar respuesta
    response = generate_response(
        instruction=message,
        max_new_tokens=200,
        temperature=0.7
    )
    return response

# Crear interfaz de chat
chat_interface = gr.ChatInterface(
    fn=chat_response,
    title="💬 MiniGPT Chat",
    description="Chatea con MiniGPT - Un modelo de 110M parámetros entrenado en español",
    examples=[
        "¿Qué es Python?",
        "Explica la fotosíntesis",
        "Escribe un haiku sobre la lluvia",
        "¿Cómo funciona Internet?"
    ],
    theme=gr.themes.Soft()
)

print("✅ Interfaz de chat creada")

In [ ]:
# Lanzar interfaz de chat
chat_interface.launch(share=True)

---

## 6. Guardar Script Standalone

Creamos un script Python que puede ejecutarse fuera de Colab.

In [ ]:
# Crear script standalone
script_content = '''
#!/usr/bin/env python3
"""
MiniGPT Instructivo - Chat Interface
Proyecto Final de Doctorado en IA/NLP

Uso:
    python chat_interface.py --mode cli      # Interfaz de línea de comandos
    python chat_interface.py --mode gradio   # Interfaz web Gradio
"""

import argparse
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# Configuración
MODEL_PATH = "./miniGPT_final"  # Ajustar según ubicación
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelo
print(f"🔄 Cargando modelo desde {MODEL_PATH}...")
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_PATH)
model = model.to(DEVICE)
model.eval()
print(f"✅ Modelo cargado en {DEVICE}")


def generate_response(
    instruction: str,
    input_text: str = "",
    max_new_tokens: int = 200,
    temperature: float = 0.7,
    top_p: float = 0.9
) -> str:
    """Genera respuesta para una instrucción."""

    if input_text:
        prompt = f"""### Instrucción:
{instruction}

### Entrada:
{input_text}

### Respuesta:
"""
    else:
        prompt = f"""### Instrucción:
{instruction}

### Respuesta:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Respuesta:" in response:
        response = response.split("### Respuesta:")[-1].strip()
    if "### Instrucción:" in response:
        response = response.split("### Instrucción:")[0].strip()

    return response


def run_cli():
    """Ejecuta interfaz CLI."""
    print("\\n" + "="*60)
    print("🤖 MINIGPT INSTRUCTIVO - Chat")
    print("   Escribe \'salir\' para terminar")
    print("="*60 + "\\n")

    while True:
        try:
            user_input = input("\\n👤 Tú: ").strip()

            if user_input.lower() == "salir":
                print("\\n👋 ¡Hasta luego!")
                break

            if not user_input:
                continue

            response = generate_response(user_input)
            print(f"\\n🤖 MiniGPT: {response}")

        except KeyboardInterrupt:
            print("\\n\\n👋 ¡Hasta luego!")
            break


def run_gradio():
    """Ejecuta interfaz Gradio."""
    import gradio as gr

    def chat_fn(message, history):
        return generate_response(message)

    demo = gr.ChatInterface(
        fn=chat_fn,
        title="🤖 MiniGPT Instructivo",
        description="Modelo GPT de 110M parámetros entrenado en español",
        examples=["¿Qué es Python?", "Explica la fotosíntesis", "Escribe un poema corto"],
    )

    demo.launch(share=True)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="MiniGPT Chat Interface")
    parser.add_argument("--mode", choices=["cli", "gradio"], default="cli",
                        help="Modo de interfaz: cli o gradio")
    args = parser.parse_args()

    if args.mode == "cli":
        run_cli()
    else:
        run_gradio()
'''

# Guardar script
with open('chat_interface.py', 'w', encoding='utf-8') as f:
    f.write(script_content)

print("✅ Script guardado: chat_interface.py")

In [ ]:
# Guardar en Google Drive
import shutil

shutil.copy('chat_interface.py', f"{project_path}/chat_interface.py")
print(f"✅ Script guardado en Google Drive: {project_path}/chat_interface.py")

---

## 7. Resumen del Proyecto Completo

In [ ]:
print("""
================================================================================
🎉 ¡PROYECTO COMPLETADO!
================================================================================

📋 TAREAS COMPLETADAS:

✅ Tarea 1: Tokenizador BPE
   - 32,000 tokens
   - Entrenado en corpus español
   - Análisis de frecuencias y merge operations

✅ Tarea 2: Comparación Encoder vs Decoder
   - Implementación de ambas arquitecturas
   - Visualización de patrones de atención
   - Justificación del diseño decoder-only

✅ Tarea 3: Entrenamiento MiniGPT
   - 110M parámetros
   - 57,471 instrucciones en español
   - 5,121 pasos de entrenamiento
   - Loss: 9.06 → 3.62 (60% reducción)

✅ Tarea 4: Evaluación Exhaustiva
   - Perplexity
   - BLEU y ROUGE
   - Benchmark personalizado
   - Tests de alucinación
   - Métrica propia (Coherence Score)
   - Análisis de errores

✅ Tarea 5: Interfaz Interactiva
   - CLI (línea de comandos)
   - Gradio (interfaz web)
   - Script standalone

================================================================================

📁 ESTRUCTURA FINAL EN GOOGLE DRIVE:

MiniGPT_Doctoral/
├── data/
│   └── processed/
│       ├── train.json
│       └── validation.json
├── tokenizer/
│   ├── bpe_tokenizer.json
│   └── hf_tokenizer/
├── miniGPT_final/
│   ├── config.json
│   ├── model.safetensors
│   └── tokenizer.json
├── checkpoints/
├── analysis/
│   ├── training_curves.png
│   ├── benchmark_results.png
│   ├── evaluation_report.txt
│   └── ...
├── logs/
└── chat_interface.py

================================================================================

📝 PARA EL INFORME TÉCNICO (20 páginas):

Parte I - Fundamentos Teóricos:
  - Usar contenido de Notebook 03 (Encoder vs Decoder)
  - Explicar arquitectura Transformer
  - Describir tokenización BPE

Parte II - Pipeline Completo:
  - Notebooks 01-04 documentan todo el proceso
  - Incluir curvas de entrenamiento
  - Mostrar ejemplos de generación

Parte III - Análisis:
  - Usar evaluation_report.txt
  - Discutir limitaciones observadas
  - Proponer mejoras futuras

================================================================================
""")

---

## ✅ ¡PROYECTO COMPLETADO!

Has completado todas las tareas del proyecto:

| Tarea | Descripción | Estado |
|-------|-------------|--------|
| 1 | Tokenizador BPE | ✅ |
| 2 | Encoder vs Decoder | ✅ |
| 3 | Entrenamiento MiniGPT | ✅ |
| 4 | Evaluación Exhaustiva | ✅ |
| 5 | Interfaz Interactiva | ✅ |

**¡Felicitaciones por completar tu proyecto doctoral!** 🎓